In [5]:
# Importando las bibliotecas necesarias
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de validación: {X_val.shape[0]}")


Tamaño del conjunto de entrenamiento: 120
Tamaño del conjunto de validación: 30


In [6]:
import torch.nn as nn

class SimpleFeedForwardNN(nn.Module):
    def __init__(self):
        super(SimpleFeedForwardNN, self).__init__()
        
        # Capa de entrada
        self.input_layer = nn.Linear(4, 10)  # 4 características de entrada y 10 neuronas en la capa oculta
        
        # Capa oculta
        self.hidden_layer = nn.Linear(10, 10)  # 10 neuronas en la capa oculta y 10 en la siguiente
        
        # Capa de salida
        self.output_layer = nn.Linear(10, 3)  # 10 neuronas en la capa oculta y 3 neuronas de salida
        
    def forward(self, x):
        # Pasar la entrada a través de la capa de entrada y luego aplicar ReLU
        x = nn.ReLU()(self.input_layer(x))
        
        # Pasar la salida anterior a través de la capa oculta y luego aplicar ReLU
        x = nn.ReLU()(self.hidden_layer(x))
        
        # Pasar la salida anterior a través de la capa de salida
        # Nota: No aplicamos Softmax aquí porque durante el entrenamiento utilizaremos
        # CrossEntropyLoss que ya aplica Softmax.
        return self.output_layer(x)
        


In [7]:
import torch
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

model = SimpleFeedForwardNN()

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_functions = {
    "CrossEntropy": nn.CrossEntropyLoss(),
    "MSELoss": nn.MSELoss(),
    "SmoothL1Loss": nn.SmoothL1Loss()
}

def evaluate_model(model, criterion, test_loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            
            if name in ["MSELoss", "SmoothL1Loss"]:
                targets = torch.nn.functional.one_hot(targets, 3).float()

            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)


for name, criterion in loss_functions.items():
    model.apply(lambda m: m.reset_parameters() if type(m) == nn.Linear else None)
    
    for epoch in range(50):
        model.train()
        total_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)

            if name in ["MSELoss", "SmoothL1Loss"]:
                targets = torch.nn.functional.one_hot(targets, 3).float()
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        test_loss = evaluate_model(model, criterion, test_loader)
        print(f"Epoch {epoch+1} [{name}] - Training Loss: {total_loss / len(train_loader):.4f}, Test Loss: {test_loss:.4f}")


Epoch 1 [CrossEntropy] - Training Loss: 1.0722, Test Loss: 1.0456
Epoch 2 [CrossEntropy] - Training Loss: 1.0607, Test Loss: 1.0338
Epoch 3 [CrossEntropy] - Training Loss: 1.0480, Test Loss: 1.0218
Epoch 4 [CrossEntropy] - Training Loss: 1.0366, Test Loss: 1.0103
Epoch 5 [CrossEntropy] - Training Loss: 1.0261, Test Loss: 0.9987
Epoch 6 [CrossEntropy] - Training Loss: 1.0166, Test Loss: 0.9871
Epoch 7 [CrossEntropy] - Training Loss: 1.0039, Test Loss: 0.9750
Epoch 8 [CrossEntropy] - Training Loss: 0.9919, Test Loss: 0.9630
Epoch 9 [CrossEntropy] - Training Loss: 0.9821, Test Loss: 0.9507
Epoch 10 [CrossEntropy] - Training Loss: 0.9718, Test Loss: 0.9377
Epoch 11 [CrossEntropy] - Training Loss: 0.9601, Test Loss: 0.9241
Epoch 12 [CrossEntropy] - Training Loss: 0.9458, Test Loss: 0.9100
Epoch 13 [CrossEntropy] - Training Loss: 0.9346, Test Loss: 0.8954
Epoch 14 [CrossEntropy] - Training Loss: 0.9186, Test Loss: 0.8804
Epoch 15 [CrossEntropy] - Training Loss: 0.9078, Test Loss: 0.8646
Epoc

In [9]:
import time
import torch.optim as optim

batch_sizes = {
    "SGD": 1,
    "Batch GD": len(train_data),
    "Mini-Batch GD": 32 
}

models = {name: SimpleFeedForwardNN() for name in batch_sizes.keys()}
optimizers = {name: optim.SGD(model.parameters(), lr=0.01) for name, model in models.items()}

for name, batch_size in batch_sizes.items():
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    model = models[name]
    optimizer = optimizers[name]
    
    print(f"Training with {name}")
    
    for epoch in range(50):
        model.train()
        start_time = time.time()
        total_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        test_loss = evaluate_model(model, nn.CrossEntropyLoss(), test_loader)
        
        print(f"Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}, Test Loss: {test_loss:.4f}, Time: {elapsed_time:.2f} seconds")

Training with SGD
Epoch 1 - Training Loss: 0.9960, Test Loss: 0.8535, Time: 0.12 seconds
Epoch 2 - Training Loss: 0.7526, Test Loss: 0.6448, Time: 0.13 seconds
Epoch 3 - Training Loss: 0.6057, Test Loss: 0.5336, Time: 0.12 seconds
Epoch 4 - Training Loss: 0.5322, Test Loss: 0.4758, Time: 0.13 seconds
Epoch 5 - Training Loss: 0.4879, Test Loss: 0.4363, Time: 0.13 seconds
Epoch 6 - Training Loss: 0.4546, Test Loss: 0.4050, Time: 0.12 seconds
Epoch 7 - Training Loss: 0.4261, Test Loss: 0.3775, Time: 0.14 seconds
Epoch 8 - Training Loss: 0.3984, Test Loss: 0.3569, Time: 0.15 seconds
Epoch 9 - Training Loss: 0.3777, Test Loss: 0.3303, Time: 0.13 seconds
Epoch 10 - Training Loss: 0.3540, Test Loss: 0.3043, Time: 0.13 seconds
Epoch 11 - Training Loss: 0.3308, Test Loss: 0.2718, Time: 0.14 seconds
Epoch 12 - Training Loss: 0.2999, Test Loss: 0.2569, Time: 0.13 seconds
Epoch 13 - Training Loss: 0.2884, Test Loss: 0.2257, Time: 0.13 seconds
Epoch 14 - Training Loss: 0.2621, Test Loss: 0.2045, Ti